In [ ]:
import ROOT
import glob
import math
import numpy as np
from IPython.display import display, Markdown
from IPython.display import display, HTML
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.checkHistograms import *

In [ ]:
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

In [ ]:
# Read the Root Files
path = "/home/berki/alice/data/2022/LHC22s/apass2/"
fileList = glob.glob(path+"*_QC.root")
fileList.sort()
#fileList = fileList[13:]
runList = [fileList[i][42:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))
#rootDataFile[0].ls()
#rootDataFile[0].ClusterQC.ls()
#rootDataFile[0].PIDQC.ls()
#rootDataFile[0].TracksQC.ls()
#runList

# Reporting on TPC QC for LHC22s [apass2]
> Link to JIRA: https://alice.its.cern.ch/jira/browse/O2-3393 <br />
> B = +0.5 T <br />
> beam = Pb-Pb <br />
> energy (sqrt) = 5.36 TeV <br />

## Runs:

| Run Number | Duration | Filling scheme                        | TPC Quality | Comment |
| ---------- | -------- | ------------------------------------- | ----------- | ------- |
| 529397     | 00:13:58 | Multi_20b_7_16_7_3bpi_8inj_PbTest2022 |  `Mixed`           |   Not perfect dEdx calibration      |
| 529399     | 00:34:48 | Multi_20b_7_16_7_3bpi_8inj_PbTest2022 |       `Mixed`      |  Missing time gain calibration          |
| 529403     | 00:53:57 | Multi_20b_7_16_7_3bpi_8inj_PbTest2022 |          `Bad`   |    Problems in sector C9     |
| 529414     | 00:33:10 | Multi_20b_7_16_7_3bpi_8inj_PbTest2022 |            `Mixed` |     Not perfect calibration       |
| **529418**     | **08:30:22** | 50ns_24b_8_24_5_8bpi_10inj_PbPbtrains |        `Mixed`     |    Missing time gain calibration        |



# Clusters QC
### Run 529403 - Problem on Sector-C 9 OROCs
Loss of CRUs during run

In [ ]:
%jsroot on
objectName = "c_N_Clusters"
c = rootDataFile[2].ClusterQC.Get(objectName)
c.Draw()

### Other runs look OK, no problems observed
As an example, plots from the golden run 529418 (over 8.5 hours)

**Further comments:**
+ OROC3 in sectors A10 and C4 noisy

In [ ]:
%jsroot on
objectName = "c_N_Clusters"
c = rootDataFile[4].ClusterQC.Get(objectName)
c.Draw()

In [ ]:
%jsroot on
objectName = "c_Q_Tot"
c = rootDataFile[4].ClusterQC.Get(objectName)
c.Draw()

In [ ]:
%jsroot on
objectName = "c_Q_Max"
c = rootDataFile[4].ClusterQC.Get(objectName)
c.Draw()

In [ ]:
%jsroot on
objectName = "c_Sigma_Time"
c = rootDataFile[4].ClusterQC.Get(objectName)
c.Draw()

In [ ]:
%jsroot on
objectName = "c_Sigma_Pad"
c = rootDataFile[4].ClusterQC.Get(objectName)
c.Draw()

In [ ]:
%jsroot on
objectName = "c_Time_Bin"
c = rootDataFile[4].ClusterQC.Get(objectName)
c.Draw()

---
## Tracks QC
### Geometrical distributions of tracks
**Phi distribution A-side tracks:**
+ Loss of tracking efficiency in sector A5
+ Hard to spot in pad-wise cluster QC plots but it that appears one of the FEC on the OROC2 in this sector might have a problem 

In [ ]:
%jsroot on
objectName="hPhiAside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,yAxisRange = [2E-3,3.5E-3], drawOption="SAME L")

#Sector boundatries
pad1.cd()
sectorBoundary = ROOT.TLine(0,hist.GetMinimum(),0,hist.GetMaximum())
sectorBoundary.SetLineStyle(2)
sectorNum = ROOT.TText(.5,.5,"test")
sectorNum.SetTextAlign(22)
sectorNum.SetTextColor(ROOT.kGray+1)
sectorNum.SetTextFont(0)
for j in range(19):
    sectorBoundary.DrawLine((j*2*math.pi)/18,hist.GetMinimum(),(j*2*math.pi)/18,hist.GetMaximum())
    if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist.GetMaximum(),str(j))
    
canvas.Draw()

**Phi distribution C-side tracks:**
+ Run 529403 as expected has losses on sector C9
+ Sector 12 also appears to have lower efficiency, **further investigation required**

In [ ]:
%jsroot on
objectName="hPhiCside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,yAxisRange = [2E-3,3.5E-3], drawOption="SAME L")

#Sector boundatries
pad1.cd()
sectorBoundary = ROOT.TLine(0,hist.GetMinimum(),0,hist.GetMaximum())
sectorBoundary.SetLineStyle(2)
sectorNum = ROOT.TText(.5,.5,"test")
sectorNum.SetTextAlign(22)
sectorNum.SetTextColor(ROOT.kGray+1)
sectorNum.SetTextFont(0)
for j in range(19):
    sectorBoundary.DrawLine((j*2*math.pi)/18,hist.GetMinimum(),(j*2*math.pi)/18,hist.GetMaximum())
    if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist.GetMaximum(),str(j))
    
canvas.Draw()

### Eta distribution
+ Enhanced bump at eta ~0.23 on A side (positive eta)
+ Was observed also in pp
+ Looking at eta vs phi no hotspots can be seen ==> general effect

In [ ]:
objectName="hEta"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,xAxisRange=[-1.1,1.1],legendNames=runList,pads=False)
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,yAxisRange = [-1.1,1.1],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

### Track properties
**Transverse momentum**
+ High pT tail slightly reduced on run 529414
+ Need to cross check

In [ ]:
objectName="hPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,log="logxy",yAxisRange = [1E-5,1E-1],legendNames=runList,pads=False)
canvas.Draw()

**Track sign**
+ Ratio as expected

In [ ]:
objectName="hSign"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False)
canvas.Draw()

In [ ]:
objectName="hQOverPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False)
canvas.Draw()

**Number of clusters per track**
+ Main peak at 130 cls
+ For pp this was observed in runs with IR>500kHz
+ Could be related to space charge distortions

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile[-5:],normalize=True,legend=True,xAxisRange = [50,160],legendNames=runList[-5:],pads=False)
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,xAxisRange = [-1.1,1.1],yAxisRange = [50,170],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logx",yAxisRange = [50,170],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

### PID
+ Gain calibration was not applied for Runs 529399 & 529418
    + dEdx BAD
+ Runs 529397 and 529414 observed to have not perfect calibration
    + Different configuration?

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC"
#quality = checkHistograms(objectName,rootDataFile,check="(histogram.Fit(\"gaus\",\"Sq\",\"\",40,60)) and 49<histogram.GetFunction(\"gaus\").GetParameter(1)<51")
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,legendNames=runList,normalize=True,log="logx",yAxisRange=[0,0.1],legend=True,pads=False)

pad1.cd()
sectorBoundary = ROOT.TLine(50,hist.GetMinimum(),50,hist.GetMaximum())
sectorBoundary.SetLineStyle(2)
sectorBoundary.SetLineWidth(4)
sectorBoundary.Draw()
    
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logy",legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logz",xAxisRange = [55,180],legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logz",legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logz",legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()

### dEdx vs p
Looks GOOD

In [ ]:
%jsroot on
objectName="hdEdxTotVsP_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logxyz",legendNames=runList,pads=True,drawOption="COLZ")
canvas.Draw()